# Detekcija i rešavanje osmosmerke

In [ ]:
from PIL import Image
import cv2
import pytesseract
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = 25, 15

In [ ]:
# poboljšanje rezolucije slike na 300 DPI
def change_resolution(image_path):
    image = Image.open(image_path)
    image.save(image_path, dpi=(300, 300))
    
# učitavanje slike
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

# konvertovanje slike u grayscale
def image_to_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# konvertovanje slike u binarnu
def image_to_bin(image_gs):
    height, width = image_gs.shape[0:2]
    image_binary = np.ndarray((height, width), dtype=np.uint8)
    return cv2.threshold(image_gs, 127, 255, cv2.THRESH_BINARY)[1]

# ispravljanje rotirane slike
def skew_correction(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    # cv2.minAreaRect() vraća vrednosti iz intervala [-90, 0)
    if angle == 0:
        return image
    elif angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REPLICATE)

# uklanjanje šumova
def smoothing_image(image):
    t1 = cv2.threshold(image, 180, 255, cv2.THRESH_BINARY)[1]
    t2 = cv2.threshold(t1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    blur = cv2.medianBlur(t2, 1, 0)
    t3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return t3

# otvaranje = erozija + dilacija
def opening(image):
    kernel = np.ones((3, 3))
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# prikaz slike
def display_image(image, color=False):
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image, 'gray')

# dobavljanje sortiranih površina unutar kontura
def get_sorted_areas(contours):
    areas = []
    for contour in contours:
        areas.append(cv2.contourArea(contour))
    areas.sort()
    return areas

# selektovanje i vraćanje okvira osmosmerke
def find_frame(image_bin, contours):
    frame_area = get_sorted_areas(contours)[-2]
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour) 
        area = cv2.contourArea(contour) 
        if area == frame_area:
            return (x, y, w, h)

# selektovanje regiona oko slova zadatih reči
def select_letters(image_bin, contours, frame):
    mask = np.zeros(image_bin.shape[:2], dtype=np.uint8)
    sorted_regions = [] 
    regions_array = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour) 
        area = cv2.contourArea(contour) # area se računa kako bi izbacili šumove
        if area > 150 and h < 100 and h > 30:
            # frame[0] je x koordinata gornjeg levog ćoška frame-a, frame[1] je y koordinata, a frame[3] je visina frame-a
            if x < frame[0] or y > frame[1] + frame[3]:
                cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 0, 0), -1)
    return mask        

# isecanje slike na veličinu frame-a
def crop_to_frame(image, frame):
    x, y, w, h = frame
    return image[y:y+h+1, x:x+w+1]

# uklanjanje slike unutar osmosmerke, ukoliko ona postoji
def remove_image(wordsearch, contours):
    mask = np.zeros(wordsearch.shape[:2], dtype=np.uint8)
    areas = get_sorted_areas(contours)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour) 
        area = cv2.contourArea(contour)
        if area == areas[-3] and areas[-4] < areas[-3]/5:
            cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 0, 0), -1)
    return mask

## Pretprocesiranje slika

In [ ]:
# TODO: odraditi za sve slike
change_resolution('data/308vx - 01.jpg')
image_color = load_image('data/308vx - 01.jpg')
image_rotated = skew_correction(image_color)
image_bin = image_to_bin(image_to_gray(image_rotated))
image_smooth = smoothing_image(image_bin)
image = opening(image_smooth)
    
display_image(image)

## Detekcija zadatih reči

In [ ]:
# prvo pronalazimo okvir osmosmerke, kako bi posmatrali samo reči izvan njega
contours = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
frame = find_frame(image, contours)

# izolujemo samo regione koji sadrže zadate reči
mask = select_letters(image, contours, frame) 
image_words = cv2.bitwise_and(image, image, mask=mask)
image_words[mask==0] = 255

display_image(image_words)

In [ ]:
alphabet = ['A', 'B', 'C', 'Č', 'Ć', 'D', 'Dž', 'Đ', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'Lj', 'M', 'N', 'Nj', 'O', 'P', 'R', 'S', 'Š', 'T', 'U', 'V', 'Z', 'Ž', '\n']

words = pytesseract.image_to_string(image_words, lang='srp_latn')
words = ''.join([c for c in words if c in alphabet]).split() # uklanjanje slova koja se ne nalaze u alfabetu

print(words)

## Detekcija slova u osmosmerci

In [ ]:
# isecanje dela slike koji sadrži osmosmerku
image_wordsearch = crop_to_frame(image, frame)
contours = cv2.findContours(image_wordsearch.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]

# uklanjanje slike rešenja unutar osmosmerke
mask = remove_image(image_wordsearch, contours)
image_wordsearch = cv2.bitwise_and(image_wordsearch, image_wordsearch, mask=mask)
image_wordsearch[mask==0] = 255

display_image(image_wordsearch)

In [ ]:
wordsearch = pytesseract.image_to_string(image_wordsearch, lang='srp_latn', config="--psm 6") # --oem 2          
wordsearch = ''.join([c for c in wordsearch if c in alphabet]).split()

print(wordsearch)

## Rešavanje osmosmerke